In [19]:
# Electricity data: Hannah Ritchie and Pablo Rosado (2020) - “Electricity Mix” Published online at OurWorldInData.org. 
# Retrieved from: 'https://ourworldindata.org/electricity-mix' [Online Resource]

# Map shape data: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

In [20]:
from bokeh.io import output_notebook, show, output_file, curdoc, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool
from bokeh.palettes import brewer
from bokeh.models import Column, Range1d
from bokeh.models.callbacks import CustomJS
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import re


In [21]:
#Read shapefile using Geopandas
shapefile = 'ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]

#Rename columns
gdf.columns = ['country', 'country_code', 'geometry']

#Drop row corresponding to 'Antarctica'
gdf = gdf[gdf['country'] != 'Antarctica']

In [22]:
# import electricity data
url_electricity_data = 'share-elec-by-source.csv'
electricity_data = pd.read_csv(url_electricity_data)

In [23]:
electricity_data = electricity_data.dropna(subset="Code")

In [24]:
electricity_data_recent = electricity_data.sort_values('Year').drop_duplicates('Code',keep='last')

In [25]:
technologies = ['Coal (% electricity)', 'Gas (% electricity)',
       'Hydro (% electricity)', 'Solar (% electricity)',
       'Wind (% electricity)', 'Oil (% electricity)',
       'Nuclear (% electricity)',
       'Other renewables excluding bioenergy (% electricity)',
       'Bioenergy (% electricity)']

In [26]:
electricity_data_recent[technologies] = electricity_data_recent[technologies]/100

In [27]:
electricity_data_recent = electricity_data_recent.rename(columns=lambda x: re.sub(' \(% electricity\)','',x))
electricity_data_recent = electricity_data_recent.rename(columns=lambda x: re.sub(' ','_',x))

In [28]:
# merge dataframes
merged = gdf.merge(electricity_data_recent, left_on = 'country_code', right_on = 'Code',how = 'left')

In [29]:
merged = merged.fillna(0)

In [30]:
from shapely.geometry import Polygon

In [31]:
merged.loc[-1] = ["Lost World","XXX",Polygon([[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]),"XXX","XXX",1,1,1,1,1,1,1,1,1,1]  # adding a row
merged.index = merged.index + 1  # shifting index
merged = merged.sort_index()  # sorting by index

In [32]:
merged.iloc[:5]

,country,country_code,geometry,Entity,Code,Year,Coal,Gas,Hydro,Solar,Wind,Oil,Nuclear,Other_renewables_excluding_bioenergy,Bioenergy
0,Lost World,XXX,"POLYGON ((0.00000 0.00000, 0.00000 0.00000, 0....",XXX,XXX,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000
1,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,FJI,2021.0,0.000000,0.000000,0.500000,0.008772,0.000000,0.359649,0.000000,0.0,0.131579
2,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",Tanzania,TZA,2021.0,0.000000,0.229829,0.366748,0.007335,0.000000,0.328851,0.000000,0.0,0.067237
3,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,CAN,2022.0,0.063814,0.105088,0.614815,0.008709,0.059726,0.004777,0.128881,0.0,0.014191


In [33]:
#Read data to json.
merged_json = json.loads(merged.to_json())

#Convert to String like object.
json_data = json.dumps(merged_json)

# Wrap the data in to GeoJSONDataSource
source = GeoJSONDataSource(geojson = json_data)

In [34]:
#Create figure object.
p = figure(title = 'Renewables in the electricity mix', height = 800 , width = 1400)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.background_fill_color = "lightgrey"

# Add patch renderer to figure
p.patches('xs','ys', source = source, fill_color = "white",
          line_color = "black", line_width = 0.25, fill_alpha = 1)

p.patches('xs','ys', source = source, fill_color = "cyan", legend_label = "Hydro",
          line_color = "black", line_width = 0.25,fill_alpha = "Hydro")

p.patches('xs','ys', source = source, fill_color = "yellow", legend_label = "Solar",
          line_color = "black", line_width = 0.25, fill_alpha = "Solar")

p.patches('xs','ys', source = source, fill_color = "magenta", legend_label = "Wind",
          line_color = "black", line_width = 0.25, fill_alpha = "Wind")

#Add hover tool
hover = HoverTool(tooltips = [ ('Country/region','@country'),
                              ('Hydro', '@Hydro{%01.1f}'),
                              ('Solar', '@Solar{%01.1f}'),
                              ('Wind', '@Wind{%01.1f}'),
                              ])

p.legend.location = "bottom_left"
p.legend.click_policy = "hide"

p.add_tools(hover)

# set a range using a Range1d
p.x_range = Range1d(-180, 180, bounds=(-180, 180))
p.y_range = Range1d(-60, 90, bounds=(-60, 90))

p.legend.title = "Technology (click to select)"

In [35]:
output_file(filename="Map13-Choropleth-Renewables.html",title="Renewable Electricity Mix")

In [36]:
show(p)